In [1]:
import numpy as np
import matplotlib.pyplot as plt

List_methode_de_flux= ["Roe","Rusanov","Godunov"]
List_condition_limite=["Neumann","Dirichlet","Periodique"]

In [ ]:

class Finite_Volume:


    def __init__(self,u0,function, derivee_de_la_function, flux_methode, film_bool =False, condition_limite = "Neumann", ghost_nodes=[0,0]):
        Valid_init = True
        self.function = function; self.derivee_de_la_function = derivee_de_la_function

        in_list =False
        for methode in List_methode_de_flux:
            if(flux_methode ==methode):  in_list =True
        if(not in_list): Valid_init = False; print("method not defined")

        in_list =False
        for condition in List_condition_limite:
            if(condition_limite ==condition): in_list =True
        if(not in_list): Valid_init = False; print("limit condition not defined")

        if(not Valid_init): f"Problème d'initialisation"

        self.flux_methode = flux_methode;   self.condition_limite = condition_limite
        self.ghost_nodes=ghost_nodes
        self.u0=u0
        self.film_bool = film_bool
    
    def sol(self,J,T,  CFL=0.95):

        dx = 1./J
        X= np.linspace(0,1,J)
        FG = np.empty_like(X) 
        FD = np.empty_like(X)
        U  = np.empty_like(X)

        Film=[]; F=[[],[]]

        U = self.u0(X)

    
        # def g(u,v):
        #     if(u<v):    return min(self.function(u),self.function(v))
        #     else :      return max(self.function(u),self.function(v))


        g = lambda u,v: 0*u + 0*v   

        if(self.flux_methode == "Godunov"): 
            g= lambda u,v : min(self.function(u),self.function(v))* (u<v) + max(self.function(u),self.function(v))*(u>=v)
        if(self.flux_methode == "Rusanov"):
            g= lambda u,v : 0.5*(self.function(u)+self.function(v))-0.5*max(np.abs(self.derivee_de_la_function(u)), np.abs(self.derivee_de_la_function(v)))*(v-u)
        if(self.flux_methode == "Roe"):
            g= lambda u,v : (0.5*(self.function(u)+self.function(v))-0.5*np.abs(self.derivee_de_la_function(u))*(v-u))*(u==v) + (self.function(u)-self.function(v))*(v<u)

        FG = np.empty_like(X) 
        FD = np.empty_like(X)

        Film=[]; F=[[],[]]  

        t=0.; n =0; dt=0.
        while t<T and n<10000:
            #calcul de dt
            vitesse = max(abs(self.derivee_de_la_function(U)))
            if(vitesse !=0) : dt = (min(CFL * dx /(2* vitesse), T-t))
            else :            dt = CFL*dx  

            if(dt<0): dt = 10e-12; print("\n !!!! dt<0 !!! \n")

            ## Calcul du flux
            for j in range(len(X)):
                
                if(j==0):   
                    if(self.condition_limite =="Dirichlet"):    FG[j] = g(self.ghost_nodes[0],       U[j])
                    elif(self.condition_limite =="Neumann"):    FG[j] = g(U[j]+self.ghost_nodes[0],U[j])
                    elif(self.condition_limite =="Periodique"): FG[j] = g(U[-1], U[j])

                else :      FG[j]= g(U[j-1],U[j])
                
                if(j==J-1):   
                    if(self.condition_limite =="Dirichlet"):    FD[j] = g(U[j], self.ghost_nodes[1])
                    elif(self.condition_limite =="Neumann"):    FD[j] = g(U[j], U[j]+self.ghost_nodes[1])
                    elif(self.condition_limite =="Periodique"): FD[j] = g(U[j], U[0])

                else :      FD[j]= g(U[j],U[j+1]) 

                
            ## eval
            if(n%(J/5)==0):
                print(sum(U)/J)
                print(dt)
                print('----------',t,'------------')
                
            ## Calcul de la solution
            for j in range(len(X)):
                U[j] = U[j] -(dt/dx)* (FD[j]-FG[j])

            if(self.film_bool): Film.append(U.copy()); F[0].append(FD.copy()); F[1].append(FG.copy())
            n+=1; t+= dt
            

        print(t,'bloop')    

        if(self.film_bool): 
            U_xt = np.zeros((len(Film),len(U)))
            for i in range(len(Film)) : U_xt[i,:] = Film[-i]
        return U_xt


In [29]:

ul=-1.;ur=1.

U0  = lambda x: ul*(x<0.5) + ur*(x>=0.5)
#U0 = lambda x: np.sin(2*np.pi*x)
f  = lambda u: 0.5*u**2
f_p= lambda u: u  

J,T = 500,2

Fv = Finite_Volume(U0,function=f,derivee_de_la_function=f_p,flux_methode="Roe",film_bool=True, condition_limite="Neumann")

U = Fv.sol(J,T)

fig, ax = plt.subplots()
im = ax.imshow(U,extent=((0,1,0,1)))
fig.colorbar(im, ax=ax, label='Interactive colorbar')
plt.show()

0.0
0.00095
---------- 0.0 ------------
8.805614015727769e-13
2.360139384751778e-10
---------- 0.011003061007480337 ------------
-3.514941316592512e-05
4.306791772444905e-18
---------- 0.01100306239084949 ------------
2629.714053163601
8.73023696780718e-26
---------- 0.011003062390849512 ------------
176271839495.09543
1.983205995077888e-33
---------- 0.011003062390849512 ------------
-5.442892131662037e+18
3.6269171458137265e-41
---------- 0.011003062390849512 ------------
2.086818317186993e+26
7.373766136665907e-49
---------- 0.011003062390849512 ------------
-3.922877321857655e+34
1.6664109980996756e-56
---------- 0.011003062390849512 ------------
-1.616981438402137e+42
3.055512632239933e-64
---------- 0.011003062390849512 ------------
6.930654719809584e+48
6.239381105423872e-72
---------- 0.011003062390849512 ------------
1.356306902183163e+57
1.4006798546177472e-79
---------- 0.011003062390849512 ------------


KeyboardInterrupt: 